In [4]:
import pandas
import re
import chromadb
import torch
import gc
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import bitsandbytes
import json

In [5]:
data = pandas.read_csv("/content/Facets_Assignment.csv")

In [ ]:
#!pip uninstall -y transformers huggingface_hub
#!pip install transformers huggingface_hub --upgrade

In [2]:
# !pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: opentelemetry-proto
    

In [6]:
def clean_facet(text):
    text = re.sub(r'^\d+[\.\s]*-*\s*', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = " ".join(text.split()).lower()
    return text

In [7]:
data['facet_clean'] = data['Facets'].apply(clean_facet)

In [8]:
data

,Facets,facet_clean
0,Risktaking,risktaking
1,Naivety,naivety
2,Acidity,acidity
3,Democratic Leadership:,democratic leadership
4,Common-sense,common sense
...,...,...
394,793. Sufi practice: Dhikr repetitions / day,sufi practice dhikr repetitions day
395,Peacefulness,peacefulness
396,Doggedness,doggedness
397,Decency,decency


In [3]:
# !pip install bitsandbytes==0.47.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.8 MB/s eta 0:00:00


In [9]:
gc.collect()
torch.cuda.empty_cache()

embed_model = SentenceTransformer('BAAI/bge-m3', device='cpu')

model_id = "mistralai/Mistral-7B-v0.3"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

gen_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="cuda:0",
    max_memory={0: "10GiB"},
)
gen_model.config.use_cache = False


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [10]:
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="facets_bench", metadata={"hnsw:space": "cosine"})

In [11]:
model = SentenceTransformer('BAAI/bge-m3')
embeddings = model.encode(data['facet_clean'].tolist())
collection.add(
    ids=[str(i) for i in data.index],
    embeddings=embeddings.tolist(),
    documents=data['facet_clean'].tolist(),
    metadatas=[{"orig": n} for n in data['Facets']]
)

Loading weights:   0%|          | 0/391 [00:00<?, ?it/s]

In [12]:
def router_agent(turn_text, top_k=10):
    query_vector = model.encode([turn_text]).tolist()
    results = collection.query(query_embeddings=query_vector, n_results=top_k)
    return results['metadatas'][0]

In [36]:
sentences = pandas.read_csv("/content/conversations.csv")

In [37]:
sentences

,Conversation
0,"""Hey! Let’s grab some coffee."""
1,"""I’m not sure this solution will work in produ..."
2,"""Can you please send the report by tonight?"""
3,"""This update completely broke my workflow."""
4,"""Thanks a lot, that was really helpful!"""
5,"""Why would anyone design it this way?"""
6,"""I feel anxious about tomorrow’s presentation."""
7,"""The system crashed again after deployment."""
8,"""Let’s take a short break and resume later."""
9,"""That joke was inappropriate for this meeting."""


In [35]:
def parse_scores(text, max_facets=10):
    """
    Extracts the first valid list of integers from model output.
    Clips values to [0,5].
    """
    match = re.search(r'(\d+\s*(?:,\s*\d+\s*){1,})', text)
    if not match:
        return []

    scores = list(map(int, match.group(1).split(',')))

    # Clip scores to valid range
    scores = [min(5, max(0, s)) for s in scores]

    return scores[:max_facets]

In [34]:
def score_turn(turn_text, selected_facets):
    facet_names = [f['orig'] for f in selected_facets]

    # Mistral specific instruction format
    prompt = f"[INST] Rate the following conversation turn on a scale of 1 to 5 for each facet provided. Provide the output as a JSON list only. [/INST]\n"
    prompt += f"Turn: {turn_text}\nFacets: {facet_names}\nOutput: ["

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = gen_model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.0,
            repetition_penalty=1.1,
            do_sample=False
        )

    decoded = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
    return decoded

In [38]:
results = []

for idx, row in sentences.iterrows():
    conversation_text = row["Conversation"].strip('"')

    selected_facets = router_agent(conversation_text)
    raw_output = score_turn(conversation_text, selected_facets)

    scores = parse_scores(raw_output, max_facets=len(selected_facets))

    facet_scores = [
        {
            "facet": f["orig"].strip(),
            "score": s
        }
        for f, s in zip(selected_facets, scores)
    ]

    results.append({
        "conversation_id": int(idx),
        "conversation_text": conversation_text,
        "num_facets_scored": len(facet_scores),
        "facet_scores": facet_scores
    })




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [39]:
output_path = "/content/conversation_facet_scores.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"Saved {len(results)} conversations to {output_path}")

Saved 50 conversations to /content/conversation_facet_scores.json
